# ImageBackend

----


In [18]:
from wholeslidedata.configuration.utils import formatted_yaml
from wholeslidedata.image.backend import WholeSlideImageBackend
from wholeslidedata import WholeSlideImage
from wholeslidedata.iterators import create_batch_iterator

For opening Whole-slide images, the wholeslidedata package relies on well-known and established python libraries such as ASAP, openslide and pyvips.
At least on of these backends needs to be installed. 

:::{seealso}
Please see the installation instructions for the image backends [here]().
:::



:::{warning}
Monochrome mask files should **not** be opened with the openslide image backend
:::



## WholeSlideImageBackend

----

In [9]:
print('Currently available backends: ', WholeSlideImageBackend.names())

Currently available backends:  ('asap', 'openslide', 'pyvips')


## WholeSlideImage

----

The image backend can be specified vai the 'backend' argument when instantiating WholeSlideImage

In [37]:
wsi = WholeSlideImage('/tmp/TCGA-21-5784-01Z-00-DX1.tif')
print('- default backend: ', wsi._backend)
wsi = WholeSlideImage('/tmp/TCGA-21-5784-01Z-00-DX1.tif', backend='asap')
print('- asap backend: ', wsi._backend)
wsi = WholeSlideImage('/tmp/TCGA-21-5784-01Z-00-DX1.tif', backend='openslide')
print('- openslide backend: ', wsi._backend)
wsi = WholeSlideImage('/tmp/TCGA-21-5784-01Z-00-DX1.tif', backend='pyvips')
print('- pyvips backend: ', wsi._backend)

- default backend:  OpenSlideWholeSlideImageBackend('/tmp/TCGA-21-5784-01Z-00-DX1.tif')
- asap backend:  <wholeslidedata.accessories.asap.backend.AsapWholeSlideImageBackend; proxy of <Swig Object of type 'MultiResolutionImage *' at 0x7ffa6ccf9540> >
- openslide backend:  OpenSlideWholeSlideImageBackend('/tmp/TCGA-21-5784-01Z-00-DX1.tif')
- pyvips backend:  <wholeslidedata.accessories.pyvips.backend.PyVipsImageBackend object at 0x7ffa98b3a340>


## Specifying Backend in user config

----

There are multiple ways to specify the image backend in the user config. The most straightforward is globally. In rare situations, you can also specify the image backend for each image separately

:::{note}
If the image backend is not specified in the config file, all images will be opened via the default image backend
:::


### global

In [33]:
user_config_global = './configs/user_config_backends-global.yml'
print(formatted_yaml(user_config_global))

wholeslidedata:
  default:
    image_backend: asap
    yaml_source:
      training:
      - wsi:
          path: /tmp/TCGA-21-5784-01Z-00-DX1.tif
        wsa:
          path: /tmp/TCGA-21-5784-01Z-00-DX1.xml
      validation:
      - wsi:
          path: /tmp/TCGA-21-5784-01Z-00-DX1.tif
        wsa:
          path: /tmp/TCGA-21-5784-01Z-00-DX1.xml
    label_map:
      stroma: 1
      tumor: 2
      lymphocytes: 3
    batch_shape:
      batch_size: 4
      spacing: 1.0
      shape: [512, 512, 3]



In [34]:
training_iterator = create_batch_iterator(mode='training', user_config=user_config_global)
validation_iterator = create_batch_iterator(mode='validation', user_config=user_config_global)
print(training_iterator.dataset['TCGA-21-5784-01Z-00-DX1']['images'][0]._backend)
print(validation_iterator.dataset['TCGA-21-5784-01Z-00-DX1']['images'][0]._backend)
training_iterator.stop()
validation_iterator.stop()

<wholeslidedata.accessories.asap.backend.AsapWholeSlideImageBackend; proxy of <Swig Object of type 'MultiResolutionImage *' at 0x7ffa97eec930> >
<wholeslidedata.accessories.asap.backend.AsapWholeSlideImageBackend; proxy of <Swig Object of type 'MultiResolutionImage *' at 0x7ffa67d9dbd0> >




### per image


In [35]:
user_config_per_image = './configs/user_config_backends-per-image.yml'
print(formatted_yaml(user_config_per_image))

wholeslidedata:
  default:
    yaml_source:
      training:
      - wsi:
          path: /tmp/TCGA-21-5784-01Z-00-DX1.tif
          image_backend: openslide
        wsa:
          path: /tmp/TCGA-21-5784-01Z-00-DX1.xml
      validation:
      - wsi:
          path: /tmp/TCGA-21-5784-01Z-00-DX1.tif
          image_backend: asap
        wsa:
          path: /tmp/TCGA-21-5784-01Z-00-DX1.xml
    label_map:
      stroma: 1
      tumor: 2
      lymphocytes: 3
    batch_shape:
      batch_size: 4
      spacing: 1.0
      shape: [512, 512, 3]



In [36]:
training_iterator = create_batch_iterator(mode='training', user_config=user_config_per_image)
validation_iterator = create_batch_iterator(mode='validation', user_config=user_config_per_image)
print(training_iterator.dataset['TCGA-21-5784-01Z-00-DX1']['images'][0]._backend)
print(validation_iterator.dataset['TCGA-21-5784-01Z-00-DX1']['images'][0]._backend)
training_iterator.stop()
validation_iterator.stop()

OpenSlideWholeSlideImageBackend('/tmp/TCGA-21-5784-01Z-00-DX1.tif')
<wholeslidedata.accessories.asap.backend.AsapWholeSlideImageBackend; proxy of <Swig Object of type 'MultiResolutionImage *' at 0x7ffa97eebbd0> >
